Успользуем даннае раннего проекта и проанализируем,как дисбаланс классов и методы его выравнивания влияют на основные метрики
и соотношения долей в матрице неточностей.

1.ОПИСАНИЕ ДАННЫХ МАССИВА

Признаки: RowNumber — индекс строки в данных CustomerId — уникальный идентификатор клиента Surname — фамилия CreditScore — кредитный рейтинг Geography — страна проживания Gender — пол Age — возраст Tenure — сколько лет человек является клиентом банка Balance — баланс на счёте NumOfProducts — количество продуктов банка, используемых клиентом HasCrCard — наличие кредитной карты IsActiveMember — активность клиента EstimatedSalary — предполагаемая зарплата

Целевой признак: Exited — факт ухода клиента

Построим и испытаем модели,используя три способа выбора гиперпараметров

In [ ]:
from __future__ import division, print_function
# отключим всякие предупреждения Anaconda
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import numpy as np

import sklearn
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn import metrics
from sklearn.metrics import f1_score,recall_score,precision_score
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import validation_curve
from sklearn.model_selection import learning_curve

from sklearn.metrics import confusion_matrix
from sklearn.metrics import plot_confusion_matrix
from sklearn.dummy import DummyClassifier
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV 
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline


!pip install optuna
import joblib 
import optuna 
from optuna.samplers import TPESampler


!pip install scikit-plot
import scikitplot as skplt

!pip install lightgbm
import lightgbm as lgb
from lightgbm import LGBMClassifier

!pip install catboost
import catboost
from catboost import Pool,CatBoost
from catboost import CatBoostClassifier
from catboost.utils import eval_metric
from catboost.utils import get_fpr_curve
from catboost.utils import get_fnr_curve
from catboost.utils import select_threshold
from catboost.utils import get_roc_curve
from catboost.utils import get_confusion_matrix

!pip3 install imblearn
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import NearMiss

In [ ]:
try:
    data = pd.read_csv('/datasets/Churn.csv')
                       
except:
                       
    data = pd.read_csv('C:/Users/анатолий/Documents/datasets/Churn.csv')
data=data.rename(columns=str.lower)#  названия столбцов перевели в строковый    
print(data.shape)    
data.head()


ИТАК видим 3 признака object фамилия,страна проживания и пол. В столбце с временем продолжительности нахождения в качестве клиента банка 909 пропущенных значений(~ 10% ) и их заполним медианой посмотрим как изменится тип данных.Остальные столбцы int и float. Посмотрим дисбаланс классов .Поработаем с кодированием категориальных признаков

In [ ]:
data['geography'].sort_values().value_counts()

In [ ]:
data[data['geography'] == 'Germany']['geography'].unique().sum()

In [ ]:
data=data.drop(['rownumber','customerid','surname'],axis=1)
data.info()
print(data.dtypes)
data.isna().sum()
display(data.head())

In [ ]:
data['tenure'] = data['tenure'].fillna(data['tenure'].median())# заполним пропуски медианой
display(data.head())
print(data.isna().sum())
data.info()
types = pd.DataFrame(data.dtypes)
print('Столбцы с категориальными данными:\n',list(types[types[0] == 'object'].index) )       

In [ ]:
data_ohe = pd.get_dummies(data,drop_first=True)

features = data_ohe.drop(['exited'],axis=1)
target = data_ohe['exited']

# оставим для теста 25% выборки а оставшиеся 75% разобьем на тренировочную и валидационную
features_train,features_test,target_train,target_test =\
                                                     train_test_split(features,target,\
                                                        test_size = 0.25,random_state = 12345,\
                                                                     stratify=target)
print('Размер тренировочной выборки',features_train.shape, target_train.shape)
print('Размер тестовой выборки',features_test.shape, target_test.shape)    # с размерностями ОК

In [ ]:
%%time

pipe_log=make_pipeline(StandardScaler(),LogisticRegression(random_state=12345))
param_grid = {
              "penalty": ["l1", "l2"],
              "C" :[0.01,0.1,1.0] ,
              "fit_intercept": [True, False],
              
              "solver": ["liblinear", "saga"]
             }

grid = GridSearchCV(LogisticRegression(multi_class="auto", max_iter=10), param_grid, cv=5)
grid = grid.fit(features_train, target_train)
grid.best_params_

In [ ]:
lgr_classifier = LogisticRegression(**grid.best_params_)
lgr_classifier.fit(features_train,target_train)
pred = lgr_classifier.predict(features_test)

print(classification_report(target_test, pred))

In [ ]:
skplt.estimators.plot_learning_curve(lgr_classifier,features_train,target_train ,
                                     cv=7, shuffle=True, scoring="f1",
                                     n_jobs=-1, figsize=(6,4), title_fontsize="large", text_fontsize="large",
                                     title=" LogisticRegression Learning Curve");

ИТАК:
Лучшая мера f1 модели на обучающей выборке:0.32
Лучший параметр регуляризации и пенальти на обучающей выборке: C=10,penalty: 'l2'}
Мера f1 модели LogisticRegression(random_state=12345,C=10,penalty=l2) на valid выборке:0.31
Логистическую регрессию отбросим из рассмотрения в виду малости меры f1.
Кривая проверки для LogisticRegression показывает что размера выборки достаточно,но повысить значение
меры f1 не получится на этой модели.
Параметр С=10 обратной регуляризации дает максимум меры f1 и видим большую погрешность на кривой проверки.
Может модель так и называется потому как здесь прям все логично 

In [ ]:
%%time
#          DecisionTreeClassifier

model_dtr = DecisionTreeClassifier()

params_grid = {'max_depth':[1,5,9,13,15]}

grid = GridSearchCV(model_dtr, param_grid=params_grid,cv=3,verbose=1)

dtr_classifier = grid.fit(features_train,target_train)

In [ ]:
print(dtr_classifier.best_score_)
print(dtr_classifier.best_estimator_.get_params())

In [ ]:
%%time
dtr_classifier =DecisionTreeClassifier(**dtr_classifier.best_params_)# определили модель с лучшими параметрами
dtr_classifier.fit(features_train,target_train)

pred_test = dtr_classifier.predict(features_test)
pred_train = dtr_classifier.predict(features_train)
print(classification_report(target_test, pred_test))

In [ ]:
# выберем гиперпараметры с помощью OPTUNA
def objective(trial):
    # Define the search space
    criterions = trial.suggest_categorical('criterion', ['gini', 'entropy'])
    max_depths = trial.suggest_int('max_depth', 1, 15, 5)
    n_estimators = trial.suggest_categorical('n_estimators', [5,15,55])

    clf = RandomForestClassifier(n_estimators=n_estimators,
                                 criterion=criterions,
                                 max_depth=max_depths,
                                 n_jobs=-1)
    f1 = cross_val_score(clf,features,target,scoring="f1").mean()

    return f1

In [ ]:
# созадим объект изучения 
study = optuna.create_study(study_name="randomForest_optimization",
                            direction="maximize",
                            sampler=TPESampler())

In [ ]:
# оптимизация обекта изучения
study.optimize(objective, n_trials=15)

In [ ]:
print('Метрика f1 выбранной лучшей модели',study.best_value)

In [ ]:
print(study.best_params)

In [ ]:
# нарисуем историю оптимизации
optuna.visualization.matplotlib.plot_optimization_history(study);

In [ ]:
# для этой оптимизации главные параметры
optuna.visualization.matplotlib.plot_param_importances(study);

In [ ]:
# таблица процесса оптимизайии
study.trials_dataframe()

In [ ]:
# как происходила тренировка до лучшего результата метрики
optuna.visualization.matplotlib.plot_edf(study, target_name="f1 score ");

In [ ]:
%%time
rfc_classifier = RandomForestClassifier(**study.best_params)# cоздание мадели с лучшими параметрами
rfc_classifier.fit(features_train,target_train)
pred_train = rfc_classifier.predict(features_train)
pred_test = rfc_classifier.predict(features_test)
print(classification_report(target_test, pred_test))


Для класса 0 с большим значением в балансе классов метрики больше,чем  для класса 1.
Получается алгоритм хорошо работает на большем количестве данных - очевидно.
Но ниже посмотрим как все проходит на тестовой выборке.


    Борьба с дисбалансом

In [ ]:
class_frequency = data['exited'].value_counts()
print('Распределение меток классов\n',class_frequency)
target.value_counts(normalize='true').plot(kind='bar',grid=True,title='Доли меток классов');

In [ ]:
#         расчет метрик с дисбалансом классов

print('Перед применением метода кол-во меток со значением 1: {}'.format(sum(target_train == True)))
print('Перед применением метода кол-во меток со значением 0: {}'.format(sum(target_train == False)))
model_random = rfc_classifier
model_random.fit(features_train,target_train)
predictions_test_random = model_random.predict(features_test)

print('recall : {:.2f}'.format(recall_score(y_true=target_test,\
                                                                y_pred=predictions_test_random)))
print('precision : {:.2f}'.format(precision_score(y_true=target_test,\
                                                                y_pred=predictions_test_random)))

print('  f1  : {:.2f}'.format(f1_score(y_true=target_test,\
                                                                y_pred=predictions_test_random)))
print(classification_report(target_test, predictions_test_random))

In [ ]:
#       Расчет метрик после балансировки классов методом SMOTE(повышение до большего)
#from imblearn.over_sampling import SMOTE
smt = SMOTE()
features_train_smote, target_train_smote = smt.fit_resample(features_train, target_train)
print('После применения метода SMOTE кол-во меток со значением 1: {}'.format(sum(target_train_smote == True)))
print('После применения метода SMOTE кол-во меток со значением 0: {}'.format(sum(target_train_smote == False)))
model_smote =  rfc_classifier
model_smote.fit(features_train_smote,target_train_smote)
predictions_smote = model_smote.predict(features_test)
print('recall : {:.2f}'.format(recall_score(y_true=target_test,\
                                                                y_pred=predictions_smote)))
print('precision : {:.2f}'.format(precision_score(y_true=target_test,\
                                                                y_pred=predictions_smote)))
print('   f1  : {:.2f}'.format(f1_score(y_true=target_test,\
                                                                y_pred=predictions_smote)))
print(classification_report(target_test, predictions_smote))

In [ ]:
#   Расчет метрик после балансировки классов методом NearMiss(с понижением до меньшего класса)


nm = NearMiss()
features_train_miss, target_train_miss = nm.fit_resample(features_train, target_train)
print('После применения метода NearMiss кол-во меток со значением 1: {}'.format(sum(target_train_miss == True)))
print('После применения метода NearMiss кол-во меток со значением 0: {}'.format(sum(target_train_miss == False)))

model_miss = rfc_classifier
model_miss.fit(features_train_miss,target_train_miss)
predictions_miss = model_miss.predict(features_test)
print('recall : {:.2f}'.format(recall_score(y_true=target_test,\
                                                                y_pred=predictions_miss)))
print('precision : {:.2f}'.format(precision_score(y_true=target_test,\
                                                                y_pred=predictions_miss)))
print('   f1  : {:.2f}'.format(f1_score(y_true=target_test,\
                                                                y_pred=predictions_miss)))
print()
print(classification_report(target_test, predictions_miss))

При уменьшении размерности f1 заметно уменьшается по двум классам,а метрика полноты recall уменьшилась для класса 0(большего),но выросла для класса 1(меньшего).Метрика точности precision для класса 0 практически не изменилась, а для класса 1 упала вдвое.
Получили:
Уменьшение размерности до меньшего класса приводит к увеличению полноты(доли ТР значений) в этом классе и понижению точности 
в этом меньшем классе - это тоже понятно.

Устранение дисбаланса до размеров большего класса(0) привело к уменьшению метрик в классе 1 по точности и росту полноты recall.
Это важно для предсказаний классов для разных задач классификации.

Далее построим ROC кривые и матрицы неточностей для разных способов балансировки

Постороим ROC кривые в зависимости от способа выравнивания дисбаланса классов

In [ ]:
rfc_classifier.fit(features_train, target_train)
lr_probs = rfc_classifier.predict_proba(features_test)# получаем предказания
lr_probs = lr_probs[:, 1]# сохраняем вероятности только для положительного исхода
fpr, tpr, treshold = roc_curve(target_test, lr_probs)
roc_auc = auc(fpr, tpr)

model_smote.fit(features_train_smote,target_train_smote)
lr_probs1 = model_smote.predict_proba(features_test)
lr_probs1 = lr_probs1[:, 1]
fpr1, tpr1, treshold1 = roc_curve(target_test, lr_probs1)
roc_auc = auc(fpr1, tpr1)

model_miss.fit(features_train_miss,target_train_miss)
lr_probs2 = model_miss.predict_proba(features_test)
lr_probs2 = lr_probs2[:, 1]
fpr2, tpr2, treshold2 = roc_curve(target_test, lr_probs2)
roc_auc = auc(fpr2, tpr2)

model_random_balance =RandomForestClassifier(random_state=12345,n_estimators=55,max_depth=11,\
                                             criterion= 'entropy',class_weight='balanced')

model_random_balance.fit(features_train,target_train).predict_proba(features_test)
lr_probs3 = model_random_balance.predict_proba(features_test)
lr_probs3 = lr_probs3[:, 1]
fpr3, tpr3, treshold3 = roc_curve(target_test, lr_probs3)
roc_auc = auc(fpr3, tpr3)


fig=plt.figure(figsize=(9,5))
ax=fig.add_subplot()        
plt.plot([0, 1], [0, 1], color='orange', linestyle='--');
roc_auc = auc(fpr, tpr)
roc_auc1 = auc(fpr1, tpr1)
roc_auc2 = auc(fpr2, tpr2)
roc_auc3 = auc(fpr3, tpr3)
ax.plot(fpr, tpr,color='blue',label='ROC кривая до балансировки (area = %0.2f)' % roc_auc)
ax.plot(fpr1, tpr1, color = 'green',label='ROC кривая с балансировкой SMOTE(area = %0.2f)' % roc_auc1) 
ax.plot(fpr2, tpr2,color='red',label='ROC кривая с балансировкой NearMiss(area = %0.2f)' % roc_auc2)
ax.plot(fpr3, tpr3,color='black',label='ROC кривая с балансировкой class_weight=balanced (area = %0.2f)' % roc_auc3)

plt.xlabel('Доля ложно-положительных прогнозов(FPR)')
plt.ylabel('Доля истинно-положительных прогнозов(TPR)')
plt.minorticks_on()
plt.grid(which='major',color='#333',linewidth=1)#большая сетка 
plt.grid(which='minor',color='#aaa',ls=':')
plt.title('ROC кривые до и после балансировки классов для модели RandomForestClassifier')

plt.legend(loc="best");


Видим,что понижение размерности до меньшего класса в дисбалансе классов уменьшает площадь под ROC кривой,тогда как 
для несбаланированного размера классов и способа class_weight='balanced' площадь почти одинаковая.
SMOTE дает чуть меньшую площадь.
Теперь рассмотрим как влияет балансировка на доли предсказываемых классов.

In [ ]:
fig,(ax1,ax2,ax3)=plt.subplots(1,3,figsize=(15,5))


confmat1=confusion_matrix(y_true=target_test,y_pred=pred_test)
print('Матрица неточностей до балансировки классов :',  confmat1)
print('До балансировки f1  : {:.2f}'.format(f1_score(y_true=target_test,\
                                                                y_pred=pred_test)))
plot_confusion_matrix(estimator=rfc_classifier, X=features_test, y_true=target_test,\
                     normalize='true', cmap='Blues',ax=ax1);
ax1.set_title('до балансировки')


confmat2=confusion_matrix(y_true=target_test,y_pred=predictions_smote)
print('Матрица неточностей для SMOTE :',  confmat2)
print('SMOTE f1  : {:.2f}'.format(f1_score(y_true=target_test,\
                                                                y_pred=predictions_smote)))
plot_confusion_matrix(estimator=model_smote, X=features_test, y_true=target_test,\
                     normalize='true',cmap='Blues',ax=ax2);
ax2.set_title('балансиpовка SMOTE')

predictions_test_balance = model_random_balance.fit(features_train,target_train).predict(features_test)
confmat3=confusion_matrix(y_true=target_test,y_pred=predictions_test_balance)
print('Матрица модели class_weight=balanced :',  confmat3)
print('class_weight=balanced  f1  : {:.2f}'.format(f1_score(y_true=target_test,\
                                                                y_pred=predictions_test_balance)))
plot_confusion_matrix(estimator=model_random_balance, X=features_test, y_true=target_test,\
                     normalize='true', cmap='Blues',ax=ax3);
ax3.set_title("class_weight='balanced' ");

In [ ]:
confmat_miss=confusion_matrix(y_true=target_test,y_pred=predictions_miss)
print('Матрица неточностей для NearMiss :',  confmat_miss)
print(' NearMiss  f1  : {:.2f}'.format(f1_score(y_true=target_test,\
                                                               y_pred=predictions_miss)))

plot_confusion_matrix(estimator=model_miss, X=features_test, y_true=target_test,\
                    normalize='true',cmap='Blues');
ax.set_title('балансировка NearMiss');


Видим что балансировка особой роли в данном случае не играет но при уменьшении дисбаланса метрика f1 падает

НО:Для способа балансировки class_weight='balanced' получили самое большое значение f1 и соответственно 
лучшее разделения по TP и TN.


Рассмотрим дополнительно бустинговую модель Catboost

АТРИБУТЫ:
best_score_ - Возвращает лучший балл модели.
classes_ - Возвращает список классов для задачи классификации.
feature_names_ — возвращает список имен компонентов.
feature_importances_ — возвращает важность каждой функции для каждого алгоритма.
learning_rate_ - Возвращает скорость обучения алгоритма.
random_seed_ — возвращает случайное семя, из которого были назначены исходные веса модели.
tree_count_ - Возвращает количество деревьев в ансамбле.
n_features_in_ — возвращает количество функций, используемых для обучения модели.
evals_result_ - Возвращает словарь оценки. Если мы предоставили оценочный набор, то результаты оценки для него будут включены.

МЕТОДИКА:
get_best_score() - Возвращает лучший балл оценщика.
get_params() - Возвращает параметры, которые были заданы в качестве словаря при создании оценщика CatBoost и их значения в виде словаря.
get_all_params() - Возвращает список всех параметров оценщика CatBoost и их значений в виде словаря.
get_cat_feature_indices() - Возвращает список индексов, который имеет категориальные признаки.
get_feature_importance() - Возвращает важность индивидуального признака в соответствии с обученной моделью.
shrink(ntree_end, ntree_start=0) - Он принимает два аргумента, которые являются конечным деревом, и начинает дерево сжимать ансамбль, чтобы включить только деревья, которые входят в этот диапазон индекса, отбрасывая все другие деревья.
set_params() - Может быть использован для задания параметров оценщика. Обратите внимание, что этот метод будет работать только перед моделью обучения.
calc_leaf_indexes(data, ntree_start=0,ntree_end=0) - Он принимает в качестве входных данных и возвращает индекс листа в каждом дереве, который использовался для прогнозирования выборки. Выходные данные этой функции будут . Он вернет индекс листьев всех деревьев для образца.n_samples x n_trees
get_leaf_values() - Возвращает фактические значения листьев деревьев в ансамбле.
get_leaf_weights() - Возвращает вес листьев для каждого листа деревьев в ансамбле.

Посмотрим работу модели без предварительной кодировки не числовых данных

In [ ]:
data['geography']=data['geography'].astype('category')
data['gender']=data['gender'].astype('category')
X = data.drop(['exited'],axis=1)
y = data['exited']

X_train,X_test,y_train,y_test = train_test_split(X,y,\
                                                        test_size = 0.25,random_state = 12345,\
                                                                     stratify=y)

print('Размер тренировочной выборки',X_train.shape, y_train.shape)
print('Размер тестовой выборки',X_test.shape, y_test.shape)    # с размерностями ОК

In [ ]:
X.info()

Рассмотрим модель с базовами параметрами

In [ ]:
booster = CatBoostClassifier(iterations=100, verbose=50, eval_metric= 'F1')
cat_features_index = [ 1, 2]
booster.fit(X_train, y_train,
            eval_set=(X_test, y_test),
            cat_features=cat_features_index,plot=True);
 
booster.set_feature_names(X_train.columns.tolist())

test_preds = booster.predict(X_test)
train_preds = booster.predict(X_train)


print("\nTest  Accuracy : %.2f"%booster.score(X_test, y_test))
print("Train Accuracy : %.2f"%booster.score(X_train, y_train))
print('Метрика f1=', f1_score(y_true=y_test,y_pred=test_preds))
print('Список всех параметров модели',booster.get_all_params())
print('Список заданных параметров',booster.get_params())


In [ ]:
booster.get_feature_importance(prettified=True)

In [ ]:
booster = CatBoostClassifier(
    random_seed=63,
    iterations=200,
    learning_rate=0.03,
)
booster.fit(
    X_train, y_train,
    cat_features=cat_features_index,
    verbose=False,
    plot=True
)

In [ ]:
#from catboost.utils import get_roc_curve
#import sklearn
#from sklearn import metrics

eval_pool = Pool(X_test, y_test, cat_features=cat_features_index)
curve = get_roc_curve(booster, eval_pool)
(fpr, tpr, thresholds) = curve
roc_auc = sklearn.metrics.auc(fpr, tpr)

In [ ]:


plt.figure(figsize=(16, 8))
lw = 2

plt.plot(fpr, tpr, color='darkorange',
         lw=lw, label='ROC curve (area = %0.2f)' % roc_auc, alpha=0.5)

plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--', alpha=0.5)

plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xticks(fontsize=16)
plt.yticks(fontsize=16)
plt.grid(True)
plt.xlabel('False Positive Rate', fontsize=16)
plt.ylabel('True Positive Rate', fontsize=16)
plt.title('Receiver operating characteristic', fontsize=20)
plt.legend(loc="lower right", fontsize=16)
plt.show()

In [ ]:
#from catboost.utils import get_fpr_curve
#from catboost.utils import get_fnr_curve

(thresholds, fpr) = get_fpr_curve(curve=curve)
(thresholds, fnr) = get_fnr_curve(curve=curve)

In [ ]:
plt.figure(figsize=(16, 8))
lw = 2

plt.plot(thresholds, fpr, color='blue', lw=lw, label='FPR', alpha=0.5)
plt.plot(thresholds, fnr, color='green', lw=lw, label='FNR', alpha=0.5)

plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xticks(fontsize=16)
plt.yticks(fontsize=16)
plt.grid(True)
plt.xlabel('Threshold', fontsize=16)
plt.ylabel('Error Rate', fontsize=16)
plt.title('FPR-FNR curves', fontsize=20)
plt.legend(loc="lower left", fontsize=16)
plt.show()

In [ ]:
# порог для заданных долей FN FP
from catboost.utils import select_threshold
print(select_threshold(model=booster, data=eval_pool, FNR=0.01))
print(select_threshold(model=booster, data=eval_pool, FPR=0.01))


Потренируем модель на скорость и определим параметры для быстрой минимизации функции потерь

In [ ]:
# определяем скорость обучения для параметров
fast_model = CatBoostClassifier(
    random_seed=63,
    iterations=150,
    learning_rate=0.1,
    boosting_type='Plain',
    bootstrap_type='Bernoulli',
    subsample=0.25,
    one_hot_max_size=25,
    rsm=0.5,
    leaf_estimation_iterations=5,
    max_ctr_complexity=1)

fast_model.fit(
    X_train, y_train,
    cat_features=cat_features_index,
    verbose=False,
    plot=True
)

In [ ]:
tunned_model = CatBoostClassifier(
    random_seed=63,
    iterations=1000,
    learning_rate=0.03,
    l2_leaf_reg=3,
    bagging_temperature=1,
    random_strength=1,
    one_hot_max_size=2,
    leaf_estimation_method='Newton'
)
tunned_model.fit(
    X_train, y_train,
    cat_features=cat_features_index,
    verbose=False,
    eval_set=(X_test, y_test),
    plot=True
)

Подберем гиперпараметры и найдем лучшую модель

In [ ]:
model = CatBoostClassifier(eval_metric= 'F1')

params = {
            'iterations':[50,100,150],
            'learning_rate':[0.01, 0.1, 0.5],
            'bootstrap_type':['Bayesian', 'No']}
            
search_results = model.grid_search(params, features_train, target_train,cv=5,stratified=True,verbose=0)



In [ ]:

model.fit(features_train, target_train,
           eval_set=(features_test, target_test),
            plot=True);
model.set_feature_names(features_train.columns.tolist()) 

test_preds = model.predict(features_test)
train_preds = model.predict(features_train)

print("\nTest  Accuracy : %.2f"%model.score(features_test, target_test))
print("Train Accuracy : %.2f"%model.score(features_train, target_train))
print('Метрика f1=', f1_score(y_true=target_test,y_pred=test_preds))
print(classification_report(target_test,test_preds ))
print("\nЛучшие выбранные параметры : ", search_results['params'])

print("Параметры заданные при создании модели : ",model.get_params())
print("\nВсе возможные параметры этой модели : ",model.get_all_params())

In [ ]:
# определили лучшие параметры
cv_results = pd.DataFrame(search_results["cv_results"])
cv_results.head()

In [ ]:
cm = get_confusion_matrix(model, Pool(features_train, target_train))
print('Матрица ошибок модели',cm)
plot_confusion_matrix(estimator=model, X=features_test, y_true=target_test,\
                    normalize='true',cmap='Blues');
plt.title('Матрица ошибок Catboost');

Результат очень хороший для TN меток в нашем случае класс 0.

In [ ]:
lr_probs = model.predict_proba(features_test)

skplt.metrics.plot_roc_curve(target_test, lr_probs,
                       title="Digits ROC Curve", figsize=(12,6));

In [ ]:
skplt.metrics.plot_precision_recall_curve(target_test, lr_probs,
                       title="Digits Precision-Recall Curve", figsize=(12,6));

Статистика КС (статистика Колмогорова-Смирнова) представляет собой максимальную разницу между кумулятивной истинно-положительной и кумулятивной ложноположительной скоростью. Он отражает способность модели отличать положительные ярлыки от отрицательных. График статистики KS предназначен только для задач двоичной классификации.
Мы впервые обучили случайный лесной классификатор на данных. Затем мы передали оригинальные метки тестов и предсказали вероятности тестирования с помощью модели случайного обучения леса, чтобы построить диаграмму статистики KS.plot_ks_statistic()

In [ ]:
predict_probas = rfc_classifier.predict_proba(features_test)
skplt.metrics.plot_ks_statistic(target_test, predict_probas, figsize=(10,6));

Диаграмма кумулятивного прироста говорит нам о проценте выборок в данной категории, которые были действительно предсказаны путем таргетинга на процент от общего числа выборок. Это означает, что когда мы взяли, что многие проценты выборок из общего процента, который мы получаем от кривой для оси Y, являются метками, которые действительно были угаданы моделью из общего числа образцов этого класса в таком количестве выборок. Пунктирная линия на графике является базовой кривой (модель случайного угадывания), и наша модель должна работать лучше, чем она, и обе кривые класса должны быть выше нее в идеале. Кумулятивная кривая прибыли предназначена только для задач двоичной классификации.

Нам нужно передать исходные метки данных и предсказанные вероятности обученной моделью, чтобы построить кумулятивную кривую прибыли

In [ ]:
skplt.metrics.plot_cumulative_gain(target_test, predict_probas, figsize=(10,6));

Диаграмма точности прогнозов выводится из кумулятивной диаграммы путем взятия отношения кумулятивного прироста для каждой кривой к базовому уровню и отображения этого соотношения на оси Y. Ось X имеет то же значение, что и приведенная выше диаграмма. Кривая подъема предназначена только для задач двоичной классификации.

Нам нужно передать его исходные метки данных и предсказанные вероятности обученной моделью, чтобы построить кривую подъема

In [ ]:
skplt.metrics.plot_lift_curve(target_test, predict_probas, figsize=(10,6));

In [ ]:
lr_probas = lgr_classifier.fit(features_train, target_train).predict_proba(features_test)
rf_probas = rfc_classifier.fit(features_train, target_train).predict_proba(features_test)
dt_probas = dtr_classifier.fit(features_train, target_train).predict_proba(features_test)
cbc_probas= model.fit(features_train, target_train).predict_proba(features_test)
probas_list = [lr_probas, rf_probas,dt_probas,cbc_probas]
clf_names = ['Logistic Regression', 'Random Forest', 'DecisionTree','Catboost']
skplt.metrics.plot_calibration_curve(target_test,
                                     probas_list,
                                     clf_names, n_bins=15,
                                     figsize=(12,6)
                                     );

Кроме логистической модели все остальные хорошо отработали по классам 